#Instalments

In [ ]:
!pip install pyod 

     |████████████████████████████████| 114 kB 25.7 MB/s 
  Created wheel for pyod: filename=pyod-0.9.8-py3-none-any.whl size=136775 sha256=51f9085f3e4e1db7a0e8f977c454674fb3d6bf42e68543d18708f50cbe088dcd
  Stored in directory: /root/.cache/pip/wheels/ba/8f/95/6cb376aec9fae09d9b1622d1662c902b522deb353cb80836a6
Successfully built pyod


In [ ]:
!pip install git+https://github.com/MaxHalford/prince.git

  Cloning https://github.com/MaxHalford/prince.git to /tmp/pip-req-build-8500cwt1
  Running command git clone -q https://github.com/MaxHalford/prince.git /tmp/pip-req-build-8500cwt1
  Created wheel for prince: filename=prince-0.7.0-py3-none-any.whl size=26169 sha256=9126f980012ed5d10ecb4f4abcb6ffaa9dc388610bb5e605ed2c32fe1d0e93b9
  Stored in directory: /tmp/pip-ephem-wheel-cache-bwi6i5cs/wheels/94/18/66/38480b5bb8b2162a521eaef3e5e88a4090198324fe65f3a152
Successfully built prince


In [ ]:
!pip install hdbscan

     |████████████████████████████████| 5.2 MB 25.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330804 sha256=371c8adf9e49c97ac68da2e882413f83edef27e59441bf8177cd7eb624dcc3a9
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef4a71e896b62f5161f2bc8a14c97a
Successfully built hdbscan


In [ ]:
# !pip install shap

# Imports

In [ ]:
from os import path, system
import numpy as np
import pandas as pd

import random
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd


# Scipy
import scipy
from scipy.stats import f_oneway
from scipy.stats import ttest_ind

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra


# sklearn
from sklearn import preprocessing
from sklearn import metrics

from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import MDS
from sklearn.manifold import Isomap
from sklearn.manifold import SpectralEmbedding
from sklearn.manifold import TSNE

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

from sklearn.feature_selection import f_classif
from sklearn.metrics import mutual_info_score, normalized_mutual_info_score
from sklearn.metrics import silhouette_samples, silhouette_score

from sklearn.neighbors import LocalOutlierFactor
from sklearn.neighbors import kneighbors_graph
from sklearn.neighbors import radius_neighbors_graph

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder


# PyOD
from pyod.models.copod import COPOD
from pyod.models.hbos import HBOS
from pyod.models.sos import SOS
from pyod.models.abod import ABOD
from pyod.models.lmdd import LMDD
from pyod.models.ocsvm import OCSVM
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.deep_svdd import DeepSVDD
from pyod.models.cof import COF
from pyod.models.pca import PCA
from pyod.models.knn import KNN
from pyod.models.iforest import IForest


# matplotlib
from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
import matplotlib.cm as cm
%matplotlib inline

# Timing
from timeit import default_timer as timer

# HDBSCAN
from hdbscan import HDBSCAN

# Functions

## Models

In [ ]:
def load_dataset(dataset_path, one_in_every, sample=False):

    if not path.exists(dataset_path):
        system('cat data/USCensus1990.data.txt.* > data/USCensus1990.data.txt')

    if sample:
      a_dataframe = pd.read_csv(dataset_path, header=0, skiprows=lambda i: i % one_in_every != 0)
    else:
      a_dataframe = pd.read_csv(dataset_path)


    # need to ignore caseid
    a_dataframe = a_dataframe.drop(columns=['caseid'])

    return a_dataframe

In [ ]:
def OutlierFilter(X_train, alg=0, outlier_frac=0.1):
  
    # Proximity-Based - Connectivity-Based Outlier Factor
    if alg == 0 or alg == 'pca' or alg == 'PCA':
        clf = PCA()

    # Linear Model - One-Class Support Vector Machines
    elif alg == 1 or alg == 'knn' or alg == 'KNN':
        clf = KNN(contamination=0.1, n_neighbors=5, method='median', radius=1.0,
                  algorithm='auto', leaf_size=30, metric='minkowski', p=2, 
                  metric_params=None, n_jobs=1)

    # AutoEncoder
    elif alg == 2 or alg == 'auto_encoder' or alg == 'AutoEncoder':
        clf = AutoEncoder(epochs=100, batch_size=32, dropout_rate=0.2, 
                          l2_regularizer=0.1, validation_size=0.1, 
                          preprocessing=True, verbose=1, random_state=None, 
                          contamination=outlier_frac)

    # Neural Network - Deep One-Class Classification
    elif alg == 3 or alg == 'DeepSVDD':
        clf = DeepSVDD(c=None, use_ae=True, hidden_neurons=None, 
                       hidden_activation='relu', output_activation='sigmoid', 
                       optimizer='adam', epochs=30, batch_size=32, 
                       dropout_rate=0.2, l2_regularizer=0.1, 
                       validation_size=0.1, preprocessing=True, verbose=1, 
                       random_state=None, contamination=outlier_frac)
        
    # Neural Network - Deep One-Class Classification
    elif alg == 4 or alg == 'iforest':
        clf = IForest(contamination=outlier_frac)

    else:
        print("Unknown outliers screening algorithm")

    X_train = X_train.astype(float)
    clf.fit(X_train)

    # y_pred = clf.predict(X_train[feat_cols].values)  # predict outliers
    # outlier_index = np.where(y_pred == 1)

    return clf  # outlier_index[0]

In [ ]:
def Clustering(X, alg=0, n_clusters=10):
  
    # K-means  - To Do: try to import and use fuzzy kmean
    if alg == 0 or alg == 'kmeans' or alg == 'KMEANS':
        clusterer = KMeans(n_clusters=n_clusters)

    # AgglomerativeClustering
    elif alg == 1 or alg == 'agg' or alg == 'AgglomerativeClustering':
        clusterer = AgglomerativeClustering(n_clusters=n_clusters)

    # DBSCAN
    elif alg == 2 or alg == 'dbscan' or alg == 'DBSCAN':
        clusterer = DBSCAN(eps=0.1, min_samples=50)

    # HDBSCAN - To Do: import and use hdbscan
    elif alg == 3 or alg == 'hdbscan' or alg == 'HDBSCAN':
        clusterer = HDBSCAN(cluster_selection_epsilon=0.3, min_cluster_size=200, 
                            min_samples=5)
    
    else:
        print("Unknown algorithm")

    result = clusterer.fit(X)
    return result  # .labels_

In [ ]:
def DimensionReduction(X, alg=0, neighbors_num=10):
  
    # PCA
    if alg == 0 or alg == 'pca' or alg == 'PCA':
        dim_reductor = PCA(n_components=10)

    # MDS
    elif alg == 1 or alg == 'mds' or alg == 'MDS':
        dim_reductor = MDS(n_components = 2, max_iter=100, n_init=1)

    # LLE - LocallyLinearEmbedding
    elif alg == 2 or alg == 'lle' or alg == 'LLE':
        dim_reductor = LocallyLinearEmbedding(n_neighbors=neighbors_num, 
                                              n_components=2, 
                                              eigen_solver='auto', 
                                              method='standard')

    # LEM (Laplacian Eigenmaps)
    elif alg == 3 or alg == 'lem' or alg == 'LEM':
        dim_reductor = SpectralEmbedding(n_neighbors=neighbors_num, 
                                         n_components=2)
        
    # Isomap
    elif alg == 5 or alg == 'isomap' or alg == 'ISOMAP':
        dim_reductor = Isomap(n_neighbors=neighbors_num, n_components=2)

    # ICA
    elif alg == 6 or alg == 'ica' or alg == 'ICA':
        dim_reductor = FastICA(n_components = 2)

    # t-SNE
    elif alg == 7 or alg == 'tsne' or alg == 'TSNE':
        dim_reductor = TSNE(n_components = 2)

    # KNN Graph
    elif alg == 8 or alg == 'tsne' or alg == 'TSNE':
        knn_graph = kneighbors_graph(X, n_neighbors=neighbors_num, 
                                     mode='connectivity', include_self=True)
        radius_knn_graph = radius_neighbors_graph(knn_graph, radius=3, 
                                                  mode='connectivity', 
                                                  include_self=True)
        dist_matrix, predecessors = dijkstra(csgraph=radius_knn_graph, 
                                             directed=False, 
                                             return_predecessors=True)

        dim_reductor = MDS(n_components=2)
        return dim_reductor.fit_transform(df[feat_cols].values)
    
    else:
        print("Unknown algorithm")

    return dim_reductor.fit_transform(df[feat_cols].values)

## Evaluation

In [ ]:
# https://towardsdatascience.com/silhouette-coefficient-validating-clustering-techniques-e976bb81d10c

# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

def silhouette_graph(X_sample, labels, n_clusters, silhouette_avg):
    sample_silhouette_values = silhouette_samples(X_sample, labels)

    fig, ax1 = plt.subplots()
    fig.set_size_inches(7, 7)
    
    y_lower = 10

    for i in range(n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )
        
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        y_lower = y_upper + 10

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
    
    ax1.set_yticks([])
    ax1.set_xticks([-0.4,-0.2,-0, 0.2, 0.4, 0.6, 0.8, 1])

    print("save fig")
    images_dir = '/content/drive/MyDrive/Unsupervised-Learning/Final-Project/'
    fig.savefig(f"{images_dir}/tryout.svg")
    # fig.savefig('tryout', format='svg', dpi=1200)
    
    return

In [ ]:
def run_experiment(X_train, y_train, n_experiments, sample_frac,
                   AD_model, outlier_frac, Cluster_models, n_clusters):
    print("Preprocessing starts")
    start = timer()
    mi_scores = {}
    silhouette_scores = {}
    for Cluster_model in Cluster_models:
      mi_scores[Cluster_model] = {}
      silhouette_scores[Cluster_model] = []
      for variable in label_col:
        mi_scores[Cluster_model][variable] = []

    filtered_data = X_train
    filtered_labels = y_train
    
    end = timer()
    print("Finished preprocessing, preprocessing time was " + str(end - start) + " seconds\n")
    
    for exp in range(n_experiments):
        start = timer()
        print("starting experiment number " + str(exp))
        
        # Use train_test_split to split x and y correspondingly
        _, X_sample, _, y_sample = train_test_split(filtered_data, filtered_labels, 
                                                    test_size=sample_frac)
        X_sample.reset_index(drop=True, inplace=True)
        y_sample.reset_index(drop=True, inplace=True)
        
        # Cluster methods
        for Cluster_model in Cluster_models:
          clusterer = Clustering(X_sample, alg=Cluster_model, 
                                 n_clusters=n_clusters)
          clusters_labels = clusterer.labels_
          cluster_num = len(np.unique(clusters_labels, return_counts=False))

          # mutual_info_score
          for variable in label_col:
            # Mutual information
            mi_variable = normalized_mutual_info_score(y_sample[variable].to_numpy(), clusters_labels)
            mi_scores[Cluster_model][variable].append(mi_variable)
            
          # Silhouette
          silhouette_avg = silhouette_score(X_sample, clusters_labels)
          silhouette_scores[Cluster_model].append(silhouette_avg)
          
          # silhouette_graph(X_sample, clusters_labels, cluster_num, silhouette_avg)
        
        end = timer()
        print("Finished experiment number " + str(exp) + ", time was " + str(end - start) + " seconds\n")


    return mi_scores, silhouette_scores

#Main

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/Eyalcohenx/UnsupervisedLearningProject
%cd /content/UnsupervisedLearningProject

Cloning into 'UnsupervisedLearningProject'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 21 (delta 6), reused 17 (delta 4), pack-reused 0
Unpacking objects: 100% (21/21), done.
Checking out files: 100% (10/10), done.
/content/UnsupervisedLearningProject


##Load data

In [ ]:
# Loading the data
data = load_dataset('data/USCensus1990.data.txt', one_in_every=10, sample=True)
print(len(data))

In [ ]:
label_col = ['dAge', 'dHispanic', 'iYearwrk', 'iSex']
all_col = data.columns.values.tolist()
feat_cols = list(set(all_col) - set(label_col))

In [ ]:
bin_col = data.columns[data.isin([0,1]).all()]
bin_feat_col = list(set(bin_col) - set(label_col))
non_bin_feat_col = list(set(feat_cols) - set(bin_feat_col))

print(bin_feat_col)
print(len(bin_feat_col))

['dIncome2', 'dIncome8', 'dIncome3', 'iMay75880', 'iRrelchld', 'iFeb55', 'iWWII', 'dIncome7', 'iSept80', 'dIncome5', 'dIncome6', 'iVietnam', 'iKorean', 'iOthrserv', 'dIncome4', 'iRownchld']
16


## One-hot encoding

In [ ]:
enc_X = OneHotEncoder(sparse=False, handle_unknown='ignore')

one_hot_data_X = enc_X.fit_transform(data)

In [ ]:
X_train = one_hot_data_X
y_train = data[label_col]

## MCA of discrete features

Some of the data is categorical, so encode to one hot, do an MCA (which is a PCA to one hot) and then run all the methods into a cluster, and so on. Record in the file whether we treated the data as categorical or not.

Run MCA on the discrete variables and PCA on the continuous variables and then connect everything.

In [ ]:
# https://github.com/MaxHalford/prince
from prince import MCA

In [ ]:
mca = MCA(
    n_components=50,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='auto',
    random_state=42
)

mca_transformed_train_data = mca.fit_transform(X_train)

In [ ]:
# mca_transformed_test_data = mca.transform(X_test)

In [ ]:
# mca.column_contributions

In [ ]:
# type(mca_transformed_train_data)
# print(mca_transformed_train_data.columns.values.tolist())

## Anomaly Detection

In [ ]:
outlier_filter = OutlierFilter(mca_transformed_train_data, alg='iforest', 
                               outlier_frac=0.005)

In [ ]:
train_anom_pred = outlier_filter.predict(mca_transformed_train_data.values)  # predict outliers
train_outlier_index = np.where(train_anom_pred == 1)[0]

In [ ]:
outlier_data = data.iloc[train_outlier_index]

In [ ]:
outlier_data_external = outlier_data[label_col]

In [ ]:
outlier_data_external.describe()

,dAge,dHispanic,iYearwrk,iSex
count,1230.000000,1230.000000,1230.000000,1230.000000
mean,4.665854,0.417886,2.905691,0.408943
std,1.558636,1.502698,2.158688,0.491839
min,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,1.000000,0.000000
50%,5.000000,0.000000,2.000000,0.000000
75%,6.000000,0.000000,5.000000,1.000000
max,7.000000,9.000000,7.000000,1.000000


## Evaluation Methods

**Mutual Information Score:**

If you know about a true division of data into labels, (not all external variables are like that) then the mutual information between the clusters that came out and the tagging of the points (according to the external variable) can be a measure of how close they are (the higher the mutual information then the labels increased our certainty and confirm the division into clusters).

In [ ]:
mi_scores1, silhouette_scores1 = run_experiment(mca_transformed_train_data, y_train, 
                                                n_experiments=20, sample_frac=0.1,
                                                AD_model='PCA', outlier_frac=0.01,
                                                Cluster_models=['kmeans','agg','hdbscan'], 
                                                n_clusters=20)

In [ ]:
mi_scores1

In [ ]:
silhouette_scores1

**ANOVA (+ t-Test)**

In [ ]:
F_anova, p_anova = f_oneway(mi_scores1['agg']['iYearwrk'], 
                            mi_scores1['kmeans']['iYearwrk'],
                            mi_scores1['hdbscan']['iYearwrk'])
# print(F_anova)
print(p_anova)  # need to be less than 0.001 to try t-Test

3.012559219313851e-17


In [ ]:
F_anova, p_anova = f_oneway(mi_scores1['agg']['dAge'], 
                            mi_scores1['kmeans']['dAge'],
                            mi_scores1['hdbscan']['dAge'])
# print(F_anova)
print(p_anova)  # need to be less than 0.001 to try t-Test

3.549468400556941e-23


In [ ]:
F_anova, p_anova = f_oneway(mi_scores1['agg']['iSex'], 
                            mi_scores1['kmeans']['iSex'],
                            mi_scores1['hdbscan']['iSex'])
# print(F_anova)
print(p_anova)  # need to be less than 0.001 to try t-Test

2.5294273607874593e-05


In [ ]:
F_anova, p_anova = f_oneway(mi_scores1['agg']['dHispanic'], 
                            mi_scores1['kmeans']['dHispanic'],
                            mi_scores1['hdbscan']['dHispanic'])
# print(F_anova)
print(p_anova)  # need to be less than 0.001 to try t-Test


0.40516801497075927


In [ ]:
F_ttest,p_ttest = ttest_ind(mi_scores1['agg']['iYearwrk'], 
                            mi_scores1['hdbscan']['iYearwrk'])

# print(F_ttest)
print(p_ttest)

2.298501191338044e-10


**Silhouette**

In [ ]:
# To do - add silhouette
# kmeans_silhouette = silhouette(KMeans, mca_transformed_train_datas)